In [ ]:
import json
import string
from typing import List
from dataclasses import dataclass
import uuid

import openai
from autogen_core import (
    RoutedAgent,
    SingleThreadedAgentRuntime,
    TopicId,
    DefaultTopicId,
    FunctionCall,
    Image,
    MessageContext,
    TypeSubscription,
    message_handler,
    type_subscription,
)
from autogen_core.models import ChatCompletionClient, SystemMessage, UserMessage, AssistantMessage, LLMMessage
from autogen_core.tools import  FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

from IPython.display import display, Markdown
from pydantic import BaseModel, Field
from rich.console import Console
from rich.markdown import Markdown as RichMarkdown
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
class GroupChatMessage(BaseModel):
    body: UserMessage

class RequestToSpeak(BaseModel):
    pass

In [6]:
class GroupBaseChatAgent(RoutedAgent):
    def __init__(self, description: str, group_topic_type: str, model_client: ChatCompletionClient, system_message: SystemMessage, topic_id: TopicId = DefaultTopicId("group_chat")):
        super().__init__(description=description)
        self._group_topic_type = group_topic_type
        self._model_client = model_client
        self._system_message = system_message
        self._chat_history: List[LLMMessage] = []

    @message_handler
    async def handle_message(self, message: GroupChatMessage, context: MessageContext) -> None:
        self._chat_history.extend([
            UserMessage(content=f"Transferred to {message.body.source}", source='system'),
            message.body,
        ])

    @message_handler
    async def handle_request_to_speak(self, message: RequestToSpeak, context: MessageContext) -> None:
        Console().print(RichMarkdown(f"### {self.id.type}"))
        self._chat_history.append(UserMessage(content=f"Transferred to {self.id.type}, adopt the persona immediately.", source="system"))

        completion = await self._model_client.create(messages=[self._system_message] + self._chat_history)
        self._chat_history.append(AssistantMessage(content=completion.content, source=self.id.type))
        assert isinstance(completion.content, str), "Completion content must be a string"

        Console().print(RichMarkdown(f"#### {self.id.type} Response"))
        Console().print(RichMarkdown(completion.content))

        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=completion.content, source=self.id.type)),
            topic_id=TopicId(self._group_topic_type, self.id.type)
        )